## kaggle快速下载docker-compose镜像

有一段时间通过梯子或docker mirror网站都遇到问题，故提供基于kaggle快速下载docker镜像的方法，供大家使用。
由于kaggle默认可以快速访问所有web地址，所以配置url地址即可(如dify)。
如果是macos或arm版linux，在确认指定镜像具有arm64版本时，设置arm=True。

In [ ]:
url = "https://raw.githubusercontent.com/langgenius/dify/main/docker/docker-compose.yaml"
arm = False

In [ ]:
import os
import subprocess
import requests

URL = (
    "https://raw.githubusercontent.com/langgenius/dify/main/docker/docker-compose.yaml"
)
CMD_TPL = "skopeo copy --insecure-policy docker://{} oci-archive:images/{}.tar"
RESTORE_TPL = "skopeo copy --insecure-policy oci-archive:{}.tar docker-daemon:{}\n"

def get_images(text=URL, stop="qdran", filter="", arm=False, online=True):
  ret = []
  restore = []
  used = []
  if arm:
    CMD_TPL = "skopeo copy --insecure-policy --override-arch=arm64 docker://{} oci-archive:images/{}.tar"
  else:
    CMD_TPL = "skopeo copy --insecure-policy docker://{} oci-archive:images/{}.tar"
  if online:
    response = requests.get(text)
    if response.status_code == 200:
      # 获取文件内容
      file_content = response.text
  else:
    file_content = text
  for line in file_content.splitlines():
    pline = line.lstrip()
    if pline.startswith("image: "):
      img = pline.split()[-1]
      if img not in used:
        img_filename = img.replace("/", "_").replace(".", "_").replace(":", "_")
        img_parts = img.split(':')
        if len(img_parts)<2:
            img = img+':latest'
        if filter:  # 如果给出过滤字符串，则进行判断
          if filter in img_filename:
            ret.append(CMD_TPL.format(img, img_filename))
            restore.append(RESTORE_TPL.format(img_filename, img))
            used.append(img)
        else:  # 否则无条件输出
          ret.append(CMD_TPL.format(img, img_filename))
          restore.append(RESTORE_TPL.format(img_filename, img))
          used.append(img)
        if stop:
          if stop in img:
              return ret, restore
  return ret, restore

def downloads_image_raw(images, restore, tar_flag=False):
  os.system("!skopeo | apt install -y skopeo")
  if not os.path.exists("images"):
    os.makedirs("images")
  print("==下载镜像文件")
  for cmd in images:
    filename = cmd.split(':')[-1]
    print(f"下载{filename} with {cmd}")
    if not os.path.exists(filename):
      subprocess.run(cmd, shell=True)
    else:
      print(f"-- 跳过，{filename}已存在")
  print("==写入import脚本")
  with open("images/import.sh", "w") as f:
      f.write("#!/bin/bash\n")
      f.writelines(restore)
  os.system("chmod +x images/import.sh && ls -la images")
  title = images[0].split("/")[-1]
  if tar_flag:
    print(f"==压缩镜像文件 {title}")
    os.system(f"tar -cvf docker-{title} images")

def downloads_image(url, arm=False, tar_flag=False):
    images, restore = get_images(url, arm=arm)  # langgenius_dify
    downloads_image_raw(images, restore, tar_flag)

downloads_image(url, arm=arm, tar_flag=False)  # AMD64架构，如果是ARM64架构，则arm=True